In [ ]:
import tdmclient.notebook
await tdmclient.notebook.start()

import cv2
import math

import numpy as np
import matplotlib.pyplot as plt
from numpy.core.fromnumeric import size
from matplotlib import colors
from asgiref.sync import sync_to_async #in order to load variables to communicate between sync and async

import optimal_path as op

from robot import Robot
from Map import Map
from Local_navigation import* 
from Mouvement import*
from vision import* 
from optimal_path import* 



map_lenght = 5
nb_of_square_by_side = 50
current = 1

#variables
global motor_left_target, motor_right_target


@tdmclient.notebook.sync_var
def motors(l_speed=500, r_speed=500, verbose=False):
    """
    Sets the motor speeds of the Thymio 
    param l_speed: left motor speed
    param r_speed: right motor speed
    param verbose: whether to print status messages or not
    """
    global motor_left_target, motor_right_target
    # Printing the speeds if requested
    if verbose:
        print("\t\t Setting speed : ", l_speed, r_speed)
    motor_left_target = l_speed
    motor_right_target = r_speed
    


@tdmclient.notebook.sync_var
def proxi():
    """
    Returns the proximity values of the Thymio 
    """
    global prox_horizontal
    return prox_horizontal


global move 
move = False #bool which indicate how our robot is moving (1: avoid an object VS 0: following the optimal path)
kalman_bool = False
#pourcentage_reduc = 1


George = Robot()
Lausanne = Map(map_lenght, nb_of_square_by_side)

#VideoCap = vision_initialization()
VideoCap=cv2.VideoCapture(0)

print('Hello World')

####################################
## INIT
####################################

# Prend une première image
ret, frame=VideoCap.read()

# Set the pourcentage value between the nb of pixel and nb of square by side
Lausanne.set_pourcentage(frame)

# Set the robot goal, position and angle
goal = init_goal(frame, Lausanne.get_pourcentage())
pos_robot, angle = update(frame, Lausanne.get_pourcentage(), kalman_bool)  # fonction qui retourne la position angle etc....
George.set_angle(angle)
George.set_goal(goal)
George.set_start_pos(pos_robot)
George.set_pos(pos_robot)
print ("goal", goal)
print ("start position", pos_robot)


####################################
## while
####################################


kalman_bool = False


print('while start')
while True:#(George.get_pos() != George.get_goal()):
    
    # Read the camera
    ret, frame=VideoCap.read()
    
    # Reupere la position et angle --> XAV
    #x_robot, y_robot,angle_robot = update(frame, Lausanne.get_pourcentage(), kalman_bool)
    pos_robot,angle_robot = update(frame, Lausanne.get_pourcentage())
    George.set_pos(pos_robot)
    George.set_angle(angle_robot)
    print ('position',pos_robot ,angle_robot)
    
    
    # Quit when we press q
    keyVal = cv2.waitKey(1) & 0xFF
    if keyVal == ord('q'):
        break
     
    sleep(0.1)

print("The End")
    
